In [1]:
import pandas as pd
import lightgbmmt as lgb
from sklearn.preprocessing import LabelEncoder
import numpy as np

加载数据

In [2]:
df=pd.read_csv("./data/train.csv")

In [3]:
df["y1"]=df["SalePrice"]

In [4]:
df=df.drop("Id",axis=1).drop("SalePrice",axis=1)

In [5]:
df=df.fillna(-1)

In [6]:
cate_cols=[col for col in  df.columns if df[col].dtype=='object']

In [7]:
for col in cate_cols:
    df[col]=LabelEncoder().fit_transform(df[col].apply(lambda x:str(x)))

In [8]:
df.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,y1
0,60,3,65.0,8450,1,0,3,3,0,4,...,0,0,0,0,0,2,2008,8,4,208500
1,20,3,80.0,9600,1,0,3,3,0,2,...,0,0,0,0,0,5,2007,8,4,181500
2,60,3,68.0,11250,1,0,0,3,0,4,...,0,0,0,0,0,9,2008,8,4,223500
3,70,3,60.0,9550,1,0,0,3,0,0,...,0,0,0,0,0,2,2006,8,0,140000
4,60,3,84.0,14260,1,0,0,3,0,2,...,0,0,0,0,0,12,2008,8,4,250000


In [9]:
trn_df=df[:1200]
val_df=df[1200:]

In [10]:
trn_y=trn_df[["y1"]]
val_y=val_df[["y1"]]

In [11]:
trn_x=trn_df.drop(["y1"],axis=1)
val_x=val_df.drop(["y1"],axis=1)

In [12]:
def jc(y,pred):
    df=pd.DataFrame({"y":y,"pred":pred})
    df=df.sort_values("pred")
    num=len(df)//10
    return df[-num:]["y"].mean()/df[0:num]["y"].mean()

baseline

In [13]:
param = {
    'num_leaves':48, 
    'objective':'regression',
    'max_depth':6,
    'learning_rate':.03,
    'max_bin':200,
    'lambda_l1':0.1,
    'lambda_l2':0.2,
    'verboses':10,
    'metrics':"rmse",
    'num_threads':4,
    'tree_learner': 'serial2'}

In [14]:
trn_data=lgb.Dataset(trn_x,label=trn_y["y1"],categorical_feature=cate_cols)
val_data=lgb.Dataset(val_x,label=val_y["y1"],categorical_feature=cate_cols)
clf=lgb.train(param,trn_data,verbose_eval=10,
                num_boost_round=200,valid_sets=[trn_data,val_data])
clf.set_num_labels(1)

/home/alei/anaconda3/lib/python3.8/site-packages/lightgbmmt/basic.py:1297: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[10]	training's rmse: 63630.2	valid_1's rmse: 56465.5
[20]	training's rmse: 51283.4	valid_1's rmse: 46026.9
[30]	training's rmse: 42493	valid_1's rmse: 38870.1
[40]	training's rmse: 35905.8	valid_1's rmse: 34372.3
[50]	training's rmse: 30888.4	valid_1's rmse: 31729.8
[60]	training's rmse: 27349.1	valid_1's rmse: 30066.1
[70]	training's rmse: 24871.5	valid_1's rmse: 29238.2
[80]	training's rmse: 23151.4	valid_1's rmse: 28894.8
[90]	training's rmse: 21930.2	valid_1's rmse: 28863.1
[100]	training's rmse: 20991.3	valid_1's rmse: 28699.6
[110]	training's rmse: 20212.4	valid_1's rmse: 28697.9
[120]	training's rmse: 19654.5	valid_1's rmse: 28774.3
[130]	training's rmse: 19130.8	valid_1's rmse: 28726.7
[140]	training's rmse: 18740.7	valid_1's rmse: 28723.6
[150]	training's rmse: 18220.1	valid_1's rmse: 28799.7
[160]	training's rmse: 17830.8	valid_1's rmse: 28811.9
[170]	training's rmse: 17498.7	valid_1's rmse: 28881.8
[180]	training's rmse: 17140.7	valid_1's rmse: 28926.5
[190]	training's rmse

In [15]:
jc(val_y["y1"],clf.predict(val_x))

inner_predict 260


3.487262909917235

自定义评估函数+earlystopping

In [17]:
param = {
    'num_leaves':48, 
    'objective':'regression',
    'max_depth':6,
    'learning_rate':.03,
    'max_bin':200,
    'lambda_l1':0.1,
    'lambda_l2':0.2,
    'verboses':10,
    'metrics':"custom",
    'num_threads':4,
    'tree_learner': 'serial2'}

In [18]:
num_labels=1

def eval_func(preds, train_data):
    labels = train_data.get_label()
    labels2 = labels.reshape((num_labels,-1)).transpose().reshape(-1)
    preds2 = preds.reshape((num_labels,-1)).transpose().reshape(-1)
    return 'jc_score', jc(labels2,preds2), True

In [19]:
trn_data=lgb.Dataset(trn_x,label=trn_y.values.reshape(-1),categorical_feature=cate_cols)
val_data=lgb.Dataset(val_x,label=val_y.values.reshape(-1),categorical_feature=cate_cols)
clf=lgb.train(param,trn_data,verbose_eval=10,feval=eval_func,early_stopping_rounds=100,
                num_boost_round=200,valid_sets=[trn_data,val_data])
clf.set_num_labels(1)

Training until validation scores don't improve for 100 rounds
[10]	training's jc_score: 3.76183	valid_1's jc_score: 3.208
[20]	training's jc_score: 3.82696	valid_1's jc_score: 3.33904
[30]	training's jc_score: 3.8675	valid_1's jc_score: 3.44613
[40]	training's jc_score: 3.91998	valid_1's jc_score: 3.47196
[50]	training's jc_score: 3.99752	valid_1's jc_score: 3.47196
[60]	training's jc_score: 4.01902	valid_1's jc_score: 3.47196
[70]	training's jc_score: 4.02293	valid_1's jc_score: 3.48568
[80]	training's jc_score: 4.03938	valid_1's jc_score: 3.48568
[90]	training's jc_score: 4.04282	valid_1's jc_score: 3.47857
[100]	training's jc_score: 4.05452	valid_1's jc_score: 3.47857
[110]	training's jc_score: 4.06565	valid_1's jc_score: 3.47857
[120]	training's jc_score: 4.08233	valid_1's jc_score: 3.44544
[130]	training's jc_score: 4.08028	valid_1's jc_score: 3.44544
[140]	training's jc_score: 4.08919	valid_1's jc_score: 3.47421
[150]	training's jc_score: 4.08644	valid_1's jc_score: 3.44686
[160]

In [20]:
jc(val_y["y1"],clf.predict(val_x))

inner_predict 260


3.4856759412170497